# MLB Stats-APIを使って試合情報を獲得する
https://github.com/toddrob99/MLB-StatsAPI/wiki

In [4]:
import requests
import pandas as pd
import statsapi
from datetime import datetime
from tqdm import tqdm

def convert_to_minutes(time_str: str) -> int:
    # '(より後ろ'が存在する場合、それを除外
    if '(' in time_str:
        time_str = time_str.split('(')[0].strip()
    hours, minutes = map(int, time_str.split(':'))
    return hours * 60 + minutes


def get_game_info(gameId: int):
    boxscore = statsapi.boxscore_data(gameId)
    try:
        attendance = next(item['value'] for item in boxscore['gameBoxInfo'] if item.get('label') == 'Att')
        attendance = int(attendance.replace('.', '').replace(',', ''))
        game_time_minutes = convert_to_minutes(next(item['value'] for item in boxscore['gameBoxInfo'] if item.get('label') == 'T').replace('.', ''))
        return attendance, game_time_minutes
    except StopIteration:
        return None, None

def add_game_info_to_dataframe(df):
    df["Attendance"] = [get_game_info(game_id)[0] for game_id in df["Game ID"]]
    df["Game Time (minutes)"] = [get_game_info(game_id)[1] for game_id in df["Game ID"]]
    
# statsapiからその試合がレギュラーシーズンかどうかを取得する
def get_game_type(game_id: int) -> str:
    game_data = statsapi.get('game', {'gamePk': game_id})
    return game_data['gameData']['game']['type']


# 初期のデータフレームを作成する関数
def create_initial_dataframe(start_date, end_date):
    BASE_URL = "http://statsapi.mlb.com/api/v1/schedule"
    params = {
        "sportId": 1,
        "startDate": start_date,
        "endDate": end_date
    }

    response = requests.get(BASE_URL, params=params)
    data = response.json()

    games = []

    for date in tqdm(data["dates"]):
        for game in date["games"]:
            game_id = game["gamePk"]
            game_type = statsapi.get('game', {'gamePk': game_id})['gameData']['game']['type']

            if game_type == 'R':  # レギュラーシーズンの試合の場合
                game_date = datetime.strptime(date["date"], "%Y-%m-%d")
                home_team = game["teams"]["home"]["team"]["name"]
                away_team = game["teams"]["away"]["team"]["name"]
                venue = game["venue"]["name"]

                games.append([game_id, game_date, home_team, away_team, venue])

    df = pd.DataFrame(games, columns=["Game ID", "Date", "Home Team", "Away Team", "Stadium Name"])
    return df

import time

def create_initial_dataframe_httperror(start_date, end_date):
    BASE_URL = "http://statsapi.mlb.com/api/v1/schedule"
    params = {
        "sportId": 1,
        "startDate": start_date,
        "endDate": end_date
    }

    response = requests.get(BASE_URL, params=params)
    data = response.json()

    games = []

    for date in tqdm(data["dates"]):
        for game in date["games"]:
            game_id = game["gamePk"]
            try:
                game_type = statsapi.get('game', {'gamePk': game_id})['gameData']['game']['type']
            except requests.exceptions.HTTPError as e:
                print(f"Error fetching data for game ID {game_id}: {e}")
                time.sleep(5)  # Wait for 5 seconds before next request
                continue

            if game_type == 'R':
                game_date = datetime.strptime(date["date"], "%Y-%m-%d")
                home_team = game["teams"]["home"]["team"]["name"]
                away_team = game["teams"]["away"]["team"]["name"]
                venue = game["venue"]["name"]

                games.append([game_id, game_date, home_team, away_team, venue])

    df = pd.DataFrame(games, columns=["Game ID", "Date", "Home Team", "Away Team", "Stadium Name"])
    return df


In [2]:
# years = [2015, 2016, 2017, 2018, 2019, 2020, 2021]
years = [2023] # アクセス集中防止のため、2年ごとに実行

# 各年でデータフレームを作成
dfs = [create_initial_dataframe_httperror(f"{year}-01-01", f"{year}-12-31") for year in years]

# 各年のデータフレームに観客者数と試合時間を追加
for df in dfs:
    add_game_info_to_dataframe(df)
    
# 各年のデータフレームをCSVファイルとして保存
for df, year in zip(dfs, years):
    df.to_csv(f"mlb_{year}.csv", index=False)



  0%|          | 1/244 [00:02<11:06,  2.74s/it]


KeyboardInterrupt: 

In [55]:
df_2017 = pd.read_csv("./data/mlb_2023.csv")
df_2017

,Game ID,Date,Home Team,Away Team,Stadium Name,Attendance,Game Time (minutes)
0,718780,2023-03-30,Washington Nationals,Atlanta Braves,Nationals Park,35756.0,187.0
1,718781,2023-03-30,New York Yankees,San Francisco Giants,Yankee Stadium,46172.0,153.0
2,718782,2023-03-30,Boston Red Sox,Baltimore Orioles,Fenway Park,36049.0,190.0
3,718777,2023-03-30,Chicago Cubs,Milwaukee Brewers,Wrigley Field,36054.0,141.0
4,718776,2023-03-30,Tampa Bay Rays,Detroit Tigers,Tropicana Field,25025.0,134.0
...,...,...,...,...,...,...,...
2471,716356,2023-10-01,Chicago White Sox,San Diego Padres,Guaranteed Rate Field,20588.0,187.0
2472,716352,2023-10-01,Kansas City Royals,New York Yankees,Kauffman Stadium,20662.0,144.0
2473,716364,2023-10-01,Milwaukee Brewers,Chicago Cubs,American Family Field,42946.0,143.0
2474,716353,2023-10-01,St. Louis Cardinals,Cincinnati Reds,Busch Stadium,44614.0,136.0


In [58]:
# df_2017をGame IDが重複しないように加工
# 日付が新しい行を残す

df_2017_dropped = df_2017.sort_values(by='Date', ascending=False).drop_duplicates(subset=['Game ID'], keep='first')

中止→ダブルヘッダー1試合目とかは欠損になってた。9/30のMets対Philliesとか。
あとは試合を途中までやったけど中止になったパターンとか。

In [65]:
df_2017_dropped

,Game ID,Date,Home Team,Away Team,Stadium Name,Attendance,Game Time (minutes)
2475,716404,2023-10-02,New York Mets,Miami Marlins,Citi Field,31097.0,131.0
2467,716360,2023-10-01,Seattle Mariners,Texas Rangers,T-Mobile Park,43997.0,121.0
2460,716367,2023-10-01,Baltimore Orioles,Boston Red Sox,Oriole Park at Camden Yards,36640.0,164.0
2461,716363,2023-10-01,San Francisco Giants,Los Angeles Dodgers,Oracle Park,38359.0,164.0
2462,716362,2023-10-01,Pittsburgh Pirates,Miami Marlins,PNC Park,22954.0,137.0
...,...,...,...,...,...,...,...
11,718778,2023-03-30,San Diego Padres,Colorado Rockies,Petco Park,45103.0,176.0
12,718769,2023-03-30,Oakland Athletics,Los Angeles Angels,Oakland Coliseum,26805.0,150.0
13,718770,2023-03-30,Los Angeles Dodgers,Arizona Diamondbacks,Dodger Stadium,52075.0,155.0
14,718767,2023-03-30,Seattle Mariners,Cleveland Guardians,T-Mobile Park,45268.0,134.0


In [ ]:
# 月ごとの試合時間をまとめたcsvを作成

In [2]:
import pandas as pd

In [4]:
df_2013 = pd.read_csv("../data/mlb_2013.csv")
df_2013

,Game ID,Date,Home Team,Away Team,Stadium Name,Attendance,Game Time (minutes)
0,346787,2013-03-31,Houston Astros,Texas Rangers,Minute Maid Park,41307.0,180.0
1,346187,2013-04-01,Washington Nationals,Miami Marlins,Nationals Park,45274.0,130.0
2,346755,2013-04-01,New York Yankees,Boston Red Sox,Yankee Stadium,49514.0,217.0
3,346189,2013-04-01,New York Mets,San Diego Padres,Citi Field,41053.0,181.0
4,346184,2013-04-01,Pittsburgh Pirates,Chicago Cubs,PNC Park,39078.0,179.0
...,...,...,...,...,...,...,...
2465,349168,2013-09-29,San Francisco Giants,San Diego Padres,AT&T Park,41495.0,197.0
2466,349162,2013-09-29,Los Angeles Dodgers,Colorado Rockies,Dodger Stadium,52396.0,188.0
2467,349169,2013-09-29,Arizona Diamondbacks,Washington Nationals,Chase Field,30390.0,149.0
2468,349173,2013-09-29,Seattle Mariners,Oakland Athletics,Safeco Field,17081.0,188.0


In [10]:
# 3月は4月に、10月は9月に合算する
def add_mar_oct_to_apr_sep(df):
    """各年の3月と10月を4月と9月に合算する。
    Returns:
        DataFrame: 3月と10月を4月と9月に合算したデータフレーム
    """
    df['Date'] = pd.to_datetime(df['Date'])
    df_mar_oct = df[(df["Date"].dt.month == 3) | (df["Date"].dt.month == 10)]
    df_mar_oct["Date"] = df_mar_oct["Date"].apply(lambda x: x.replace(month=4, day=min(x.day, 30)) if x.month == 3 else x.replace(month=9, day=min(x.day, 30)))
    df = pd.concat([df, df_mar_oct])
    df = df[df["Date"].dt.month != 3]
    df = df[df["Date"].dt.month != 10]
    return df

In [11]:
df_2013_ = add_mar_oct_to_apr_sep(df_2013)
print(len(df_2013_))
len(df_2013)

/var/folders/yq/lx5qv2_9299_1q6b2jtwqzn40000gn/T/ipykernel_56054/1337965154.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mar_oct["Date"] = df_mar_oct["Date"].apply(lambda x: x.replace(month=4, day=min(x.day, 30)) if x.month == 3 else x.replace(month=9, day=min(x.day, 30)))


2470

In [ ]:
# データフレームを渡すと、そのデータフレームの月毎の平均試合時間をまとめて返す関数
def get_monthly_game_time_averages(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Month'] = df['Date'].dt.month
    return df.groupby('Month')['Game Time (minutes)'].mean()